<a href="https://colab.research.google.com/github/Ria-Chaudhuri/Evaluation/blob/main/UnitTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PreReq- please install the following packages
#!pip install apache_beam
#!pip install apache-beam[gcp]

In [ ]:
#Imported Libraries and dependencies
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from datetime import datetime
import json
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to

#ParDo Transforms 
class TransactionAmountGreaterThan20(beam.DoFn):
  def process(self, element):
     if float(element[3])>20.00:
          yield element

class ConvertToDateWithTransaction(beam.DoFn):
  def process(self, element):
    date=datetime.strptime((element[0]), "%Y-%m-%d %H:%M:%S %Z").date()
    result=[(date.strftime('%Y-%m-%d'),float(element[3]))]
    return result

#Composite Transform for allplying all the transformation
@beam.ptransform_fn
def TransactionsWithAllTransforms(pcoll):
  return (
      pcoll
                    | 'Split the File'>>beam.Map(lambda x:x.split(","))
                    |'Transactions Greater Than 20  ' >> beam.ParDo(TransactionAmountGreaterThan20())
                    | 'Transactions after 2010  ' >> beam.Filter(lambda record: datetime.strptime((record[0]), "%Y-%m-%d %H:%M:%S %Z").year >2010)
                    | 'Convert To Date With Transactions' >> beam.ParDo(ConvertToDateWithTransaction())
                    | 'Sum values per key' >> beam.CombinePerKey(sum)
                    | 'FormatOutput' >> beam.Map(json.dumps)
         )

pipe= beam.Pipeline()

with beam.Pipeline(options=PipelineOptions()) as pipeline:
  lines = (pipeline | 'ReadMyFile' >> beam.io.ReadFromText('gs://cloud-samples-data/bigquery/sample-transactions/transactions.csv',skip_header_lines=True)
                    |  'Apply All the Transforms'>> TransactionsWithAllTransforms()
                    |beam.Map(print)
                    #|  'Write Output' >> beam.io.WriteToText( file_path_prefix='result.json.gz',header='date, total_amount') 
  )

In [ ]:
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to
from apache_beam.dataframe.io import read_csv
import unittest


class CountTest(unittest.TestCase):

  def test_count(self):
    # Create a test pipeline.
    with TestPipeline() as p:

      # Create an input PCollection.
      input = p | 'ReadMyFile' >> beam.io.ReadFromText('SampleData.csv',skip_header_lines=True)
                 

      # Apply the Count transform under test.
      output = input | TransactionsWithAllTransforms()
                      
                          

      # Assert on the results.
      assert_that(
       output,
        equal_to([['2018-08-09', '100.99'], ['2019-01-09', '547534.23']]))   
            
unittest.main(argv=[''], verbosity=2, exit=False)